OS: Ubuntu 22.04LTS\
Kernel: Python (gguf)

1. Download model

In [15]:
# 모델 이름 지정
# model_id = "sh2orc/Llama-3.1-Korean-8B-Instruct"
# model_name = "Llama-3.1-Korean-8B-Instruct"

model_id = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"
model_name = "EXAONE-3.0-7.8B-Instruct"

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login
import os
import json
import shutil

# WARNING: download new key.json in notion
key_path = os.path.abspath(os.getcwd()+"../../../key/key.json")
with open(key_path, 'r', encoding='utf-8') as f:
        key_data = json.load(f)

login(token=key_data["HUGGING-FACE"])

# 모델과 토크나이저 로드
cache_path = os.path.abspath(os.getcwd() + "/cache")
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, cache_dir=cache_path)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, cache_dir=cache_path)

# 로컬 디렉토리에 모델 저장
save_directory = os.path.abspath(os.getcwd() + "./save")
save_file = os.path.join(save_directory, model_name)
if not os.path.exists(save_directory):
        os.mkdir(save_directory)
model.save_pretrained(save_file)
tokenizer.save_pretrained(save_file)

# 다운로드 캐시 삭제
if os.path.exists(cache_path):
    shutil.rmtree(cache_path)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/abcd/.cache/huggingface/token
Login successful


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

configuration_exaone.py:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py:   0%|          | 0.00/81.1k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

2. Convert into Float16 gguf

In [17]:
!python llama.cpp/convert_hf_to_gguf.py ./{model_name}/ --outfile ./{model_name}-fp16.gguf


2724.80s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


INFO:hf-to-gguf:Loading model: EXAONE-3.0-7.8B-Instruct
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00007.safetensors'
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float32 --> F16, shape = {4096, 1024}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float32 --> F16, shape = {4096, 4096}
INFO:hf-to-gguf:blk.0.attn_q.weight,         torch.float32 --> F16, shape = {4096, 4096}
INFO:hf-to-gguf:blk.0.attn_v.weight,         torch.float32 --> F16, shape = {4096, 1024}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float32 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float32 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float32 --> F16, shape = {4096, 14336}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float32 --> F16, s

3. Quantize

In [20]:

# quantize = "Q8_0"
quantize = "Q4_K_M"

In [21]:
!llama.cpp/llama-quantize ./{model_name}-fp16.gguf ./{model_name}-{quantize}.gguf {quantize}

2881.58s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


main: build = 3608 (50addec9)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing './EXAONE-3.0-7.8B-Instruct-fp16.gguf' to './EXAONE-3.0-7.8B-Instruct-Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 28 key-value pairs and 291 tensors from ./EXAONE-3.0-7.8B-Instruct-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = exaone
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = EXAONE 3.0 7.8B Instruct
llama_model_loader: - kv   3:                       general.organization str              = LGAI EXAONE
llama_model_loader: - kv   4:                           general.finetune str              = Instruct
llama_model_loader